In [ ]:
using Transformation, Plots, Logging
disable_logging(LogLevel(1000));
include("graphics.jl");

# Представление положения
В рамках данного пособия положение (смещение и ориентация) объекта (или системы координат) описывается посредством связки _вектор_ и _кватернион_.

## Вектор
Вектор описывает перемещение в трехмерном пространстве и содержит три числа
$$ v = [x, y, z] $$

In [ ]:
v = Vec(1, 2, 3)

In [ ]:
v.x

In [ ]:
v.y

In [ ]:
v.z

In [ ]:
a = Vec(1, 2, 3);
b = Vec(2, 3, 0);

Вектора можно _складывать_ и _вычитать_

In [ ]:
a + b

In [ ]:
a - b

Вектора можно умножать _скалярно_

In [ ]:
dot(a, b)

Или брать _векторное_ произведение (полезно для определения перпендикуляра к двум векторам).

In [ ]:
cross(a, b)

In [ ]:
x = Vec(1, 0, 0)
y = Vec(0, 1, 0)
cross(x, y)

Так же вектор можно умножеть на число

In [ ]:
5 * Vec(1, 2, 3)

У вектора есть _норма_ (_длина_)

In [ ]:
norm(Vec(3, 4, 0))

Вектор можно _нормировать_ (находить _колинеарный_ вектор с длиной `1`)

In [ ]:
unit(Vec(3, 4, 0))

Для двух векторов $s$ и $e$ легко определить промежуточное положение $i$ по коэффициенту интерполяции $t$
$$ i = s + t (e - s) $$

In [ ]:
s = Vec(1, 2, 7)
e = Vec(-4, 3, 0)

@gif for t = 0:0.01:1
    i = s + t * (e - s)
    
    p = plot(
        xlims = (-4, 4),
        ylims = (-4, 4),
        zlims = (0, 8)
    )
    plot!(p, [0, s.x], [0, s.y], [0, s.z], color = :red, label = "From")
    plot!(p, [0, e.x], [0, e.y], [0, e.z], color = :green, label = "To")
    plot!(p, [0, i.x], [0, i.y], [0, i.z], color = :black, label = "Interpolated")
end

## Кватернион
_Кватернион_ - квазикомплексное число вида $w + ix + jy + kz$, у него есть одна действительная ($w$) и три мнимые части ($x$, $y$, $z$).
На практике можно просто представить его как структуру из четырех чисел
$$ q = [w, x, y, z] $$

In [ ]:
q = Quat(1, 2, 3, 4)

In [ ]:
q.w

In [ ]:
q.x

In [ ]:
q.y

In [ ]:
q.z

In [ ]:
q = Quat(0, 0, 1, 0);
p = Quat(0, 1, 0, 0);

Кватернионы можно складывать и вычитать поэлементно

In [ ]:
q + p

In [ ]:
q - p

Умножение кватернионов более сложно, оно базируется на свойствах квазикомплексных чисел

In [ ]:
q * p

Умножени кватернионов некоммутотивно

In [ ]:
p * q

У кватерниона есть _норма_ (_длина_)

In [ ]:
norm(Quat(0, 3, 0, 4))

Кватернион с длиной `1` называются _версор_

In [ ]:
norm(Quat(0, sqrt(2)/2, sqrt(2)/2, 0))

_Версор_ может применяться для описания вращения.
Для того чтобы описать вращение с помощью версора нужно знать угол поворота $\alpha$ (в радианах) и ось вращения (вектор) $v$
$$ \left[\cos\frac{\alpha}{2}, \sin\frac{\alpha}{2} v_x,  \sin\frac{\alpha}{2} v_y,  \sin\frac{\alpha}{2} v_z \right] $$

In [ ]:
Quat(pi/2, Vec(1, 0, 0))

Версор `[1, 0, 0, 0]` описывает нулевой поворот

In [ ]:
Quat(0, Vec(1, 0, 0))

_Комплексно сопряженный_ версор описывает обратный поворот

In [ ]:
q = Quat(0, 0, 1, 0)
conj(q)

Версор можно использовать для _поворота_ вектора. Внутри эта операция построена на основе сэндвич оператора

In [ ]:
q = Quat(pi / 2, Vec(0, 0, 1))
v = Vec(1, 2, 3)
q * v

Для двух версоров можно `q` и `p` найти промежуточный версор `i` по параметру интерполяции `t` такой, что он будет находится на кратчайшем пути поворота

In [ ]:
s = Quat(1, 0, 0, 0)
e = Quat(pi / 3, Vec(-1, 1, 1))

@gif for t = 0:0.01:1
    i = slerp(s, e, t)
    
    p = plot(
        xlims = (-1, 1),
        ylims = (-1, 1),
        zlims = (-1, 1)
    )
    plotAxis!(p, Transf(Vec(), s), 0.9)
    plotAxis!(p, Transf(Vec(), e), 0.9)
    plotAxis!(p, Transf(Vec(), i), 1)
end

## Положение
Положение описывается _вектором_ $v$ и _кватернионом_ $q$, соответстующих _последовательным_ перемещению и повороту
$$ [v, q] $$

In [ ]:
Transf(
    Vec(1, 2, 3),
    Quat(1, 0, 0, 0)
)

Имея положение можно определить составляющие его вектор и кватернион

In [ ]:
t = Transf(Vec(1, 2, 3), Quat(0, 0, 1, 0))

In [ ]:
t.v

In [ ]:
t.q

Если дано положение `t2` в локальной системе координат `t1`, то можно определить его положение в глобальной системе координат

In [ ]:
t1 = Transf(Vec(10, 20, 30), Quat(pi / 2, Vec(1, 0, 0)))
t2 = Transf(Vec(1, 2, 3), Quat(1, 0, 0, 0))
t1 + t2

Также можно определить положение вектора заданого в локальной системе координат

In [ ]:
t = Transf(Vec(10, 20, 30), Quat(pi / 2, Vec(0, 1, 0)))
v = Vec(1, 2, 3)
t + v

Преобразования можно использовать для описания кинематических цепей

In [ ]:
t0 = Transf()
t1 = t0 + Transf(Vec(0, 0, 1), Quat(pi/4, Vec(1, 0, 0)))
t2 = t1 + Transf(Vec(0, 0, 1))
[
    "начало" t0 ;
    "сустав" t1 ;
    "конец" t2
]

Кинематическую цепь можно отобразить

In [ ]:
p = plot(
    xlims = (-1, 1),
    ylims = (-1, 1),
    zlims = (0, 2)
)

plotChain!(p, [t0, t1, t2])